In [ ]:
!pip install torchsde
import torch
import soundfile as sf
from diffusers import StableAudioPipeline
from huggingface_hub import login
from IPython.display import Audio  # Import the Audio module for playback
from google.colab import userdata

READ = userdata.get('READ')

login(token=(READ));

pipe = StableAudioPipeline.from_pretrained("stabilityai/stable-audio-open-1.0", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

# Define unique fictional animal sound prompts
animal_sounds = [
    ("Roaring deep call of a Drakonis, a legendary dragon-bird hybrid", 5.0),
    ("High-pitched whistling of a Sylphfox, a magical fox that sings at dawn", 10.0),
    ("Deep underwater growl of a Leviapod, a giant alien sea creature", 15.0),
    ("Ethereal humming of a Starphoenix, a cosmic bird floating in space", 20.0)
]

# Generate and save audio files
for i, (prompt, duration) in enumerate(animal_sounds):
    print(f"Generating sound {i+1}: {prompt} ({duration}s)...")

    generator = torch.Generator("cuda")  # Set generator for reproducibility
    audio = pipe(
        prompt=prompt,
        negative_prompt="Low quality, distorted, unclear",
        num_inference_steps=200,
        audio_end_in_s=duration,
        num_waveforms_per_prompt=1,
        generator=generator,
    ).audios[0].T.float().cpu().numpy()

    # Save audio file
    filename = f"animal_sound_{i+1}.wav"
    sf.write(filename, audio, pipe.vae.sampling_rate)
    print(f"Saved: {filename}")

    # Play audio
    display(Audio(filename))


In [ ]:
# Access your secret keys via
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPEN_API_KEY')

# Import OpenAI API and set up the key
from openai import OpenAI
client = OpenAI(api_key=(OPENAI_API_KEY))

# Define a structured prompt
prompt = """
In a world where animals communicate with each other using their own unique language,
here are some common animal phrases:

1. "Waa-woof-rahh!" - The dog says this when it feels excited and wants to play.
2. "Meow-pss!" - A cat says this when it is annoyed but still curious.
3. "Chirp-chirp, chirp!" - The bird uses this phrase when it sees something interesting.
4. "Grrr-hmm!" - The bear growls this when it feels territorial.
5. "Moo-woo-rahh!" - The cow says this when calling for its herd.
6. "Roo-wuff!" - The wolf says this when it's ready to hunt in a group.
7. "Hissss-snarl!" - The snake makes this sound when it feels threatened.
8. "Quack-a-quack!" - The duck uses this when it’s trying to get attention from its fellow ducks.
9. "Squeak-squeak!" - The mouse says this when it’s warning others of a nearby predator.
10. "Oink-oink-hee!" - The pig uses this when it’s happy to see food.
11. "Ribbit-ribbit!" - The frog says this when it’s calling for a mate.
12. "Baa-aa!" - The sheep uses this to communicate with the flock.
13. "Buzz-buzz!" - The bee makes this sound when it’s about to gather nectar.
14. "Caw-caw!" - The crow uses this when it wants to alert others of danger.
15. "Grr-raa!" - The lion roars this when asserting dominance over its territory.
16. "Trill-trill!" - The bat uses this phrase when it’s echo-locating its way through the night.

Now, continue the list with more fictional animal phrases:
"""

# Generate response
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": "You are a creative AI generating animal sounds."},
              {"role": "user", "content": prompt}],
    max_tokens=450,
    temperature=0.7,
    top_p=1.0
)

# Extract the generated content
generated_text = completion.choices[0].message.content

# Extract only the sounds from the response
import re
sounds = re.findall(r'"(.*?)"', generated_text)  # Finds all text inside quotes

# Save the sounds to a .txt file as an array
file_path = "/content/animal_language.txt"

with open(file_path, "w") as file:
    file.write(str(sounds))  # Write the array to the file

print(f"Sounds saved to: {file_path}")
